In [10]:
# If needed, install Great Expectations first:
# !pip install great_expectations --quiet

import pandas as pd
import great_expectations as ge
from great_expectations.checkpoint import SimpleCheckpoint
from great_expectations.core.batch import RuntimeBatchRequest

# Create sample DataFrame
data = {
    "Product": ["A", "B", "C", "D"],
    "Sales": [100, 200, None, 400]
}
df = pd.DataFrame(data)

# Create an in-memory DataContext (uses default configuration in memory)
context = ge.get_context()

# Add a pandas datasource (Runtime Data Connector)
datasource_config = {
    "name": "my_pandas_datasource",
    "class_name": "Datasource",
    "execution_engine": {"class_name": "PandasExecutionEngine"},
    "data_connectors": {
        "default_runtime_data_connector": {
            "class_name": "RuntimeDataConnector",
            "batch_identifiers": ["default_identifier_name"],
        }
    },
}

context.add_datasource(**datasource_config)

# Prepare a RuntimeBatchRequest using the DataFrame
batch_request = RuntimeBatchRequest(
    datasource_name="my_pandas_datasource",
    data_connector_name="default_runtime_data_connector",
    data_asset_name="my_asset",
    runtime_parameters={"batch_data": df},
    batch_identifiers={"default_identifier_name": "default_id"},
)

# Create or get expectation suite
expectation_suite_name = "my_suite"
try:
    suite = context.get_expectation_suite(expectation_suite_name)
except Exception:
    suite = context.create_expectation_suite(expectation_suite_name, overwrite_existing=True)

# Get Validator for the batch
validator = context.get_validator(batch_request=batch_request, expectation_suite_name=expectation_suite_name)

# Add expectation: no nulls in "Sales"
validator.expect_column_values_to_not_be_null("Sales")

# Save the expectation suite
validator.save_expectation_suite()

# Validate the batch data
results = validator.validate()

print("Validation results:")
print(results)

# Print rows where Sales is missing
print("\nRows with missing 'Sales':")
print(df[df["Sales"].isnull()])

ImportError: cannot import name 'SimpleCheckpoint' from 'great_expectations.checkpoint' (/home/vscode/.local/lib/python3.10/site-packages/great_expectations/checkpoint/__init__.py)